In [1]:
import requests
import datetime
import pandas as pd
import json
import os

### Readme

This script contains several constituent functions which are run together in the bottom cell. Make sure to run all cells prior to running the bottom. There is an explanation above each cell as to what that function does and what inputs it takes. In summary, the whole thing is designed to: 

    1) log into the Aeroqual Cloud API
    2) check for first date data are needed or missing for the monitor
    3) request data from date identified in step 2 up to yesterday in one-week intervals
    4) concatenate data for each monitor and write to csv
    5) concatenate individual monitors into one master dataset

Login to aeroqual cloud. Requires inputs of username and password. This requires either Lee Ann or Boris' credentials to login to the cloud

In [2]:
def create_aqc_login(username, password):
    
    credentials = {'UserName': username, 'Password': password}
    
    aqc_login = requests.post('https://cloud.aeroqual.com/api/account/login/', data = credentials)
    
    print('Login status:', aqc_login)
    
    return(aqc_login)

Get list of available instruments. This will return all instruments with data available on the cloud for any period of time and any pollutants. 

In [3]:
def get_instruments_available(aqc_login):
    
    available_aqys = requests.get('https://cloud.aeroqual.com/api/instrument', cookies = aqc_login.cookies)
    
    print('Instrument list request status:', available_aqys)
    
    return(available_aqys.json())

All functions below this one are intended to work on one AQY at a time. Easiest use case is to iterate through the AQY list generated in get_instruments_available. This one gets the date the desired AQY was deployed as a first theoretically possible measurement.

In [4]:
def get_deployment_date(aqy_id):
    
    deployment_network = pd.read_csv('monitor_deployment.txt', sep = '\t')
    deployment_aqy = deployment_network[deployment_network['Monitor ID'] == aqy_id]
    
    deployment_date = deployment_aqy.iloc[0]['Deployment date']
    deployment_split = str.split(deployment_date, '-')
    
    deployment_date_dt = datetime.date(int(deployment_split[0]), int(deployment_split[1]), int(deployment_split[2]))
    
    return(deployment_date_dt)

Gets a list of all existing days in the dataset based on the existing filepath. Feeds into _find_date_to_start_ and is used to identify gaps in data.

In [5]:
def get_existing_dates_in_data(existing_file_path):
    
    existing_data = pd.read_csv(existing_file_path)
    existing_data['date_observed'] = pd.to_datetime(existing_data['Time'].str[0:11])
    
    existing_dates = pd.DataFrame(set(existing_data['date_observed'])).rename(columns = {0: 'date_observed'}).sort_values('date_observed')
    existing_dates['time_dif'] = existing_dates['date_observed'].diff()
    existing_dates = existing_dates.dropna()
        
    return(existing_dates)   

Uses existing days in data and set of all possible days to find first day not in observed data that should be. Returns this as first data gap.

In [6]:
## TO DO - ONLY LOOK FOR GAPS IN LAST 6-8 WEEKS TO IMPROVE RUN TIME WHILE BACKFILLING
def find_first_data_gap(existing_dates):
    
    dates_observed = existing_dates['date_observed']
    dates_observed_dt = set([datetime.date(date_observed.year, date_observed.month, date_observed.day) for date_observed in dates_observed])

    first_date_observed = existing_dates['date_observed'][0]
    yesterday = pd.Timestamp.today() - pd.Timedelta(value = 1, unit = 'D')
    dates_possible = pd.date_range(first_date_observed, yesterday)
    
    dates_missing = set(dates_possible.date)
    dates_missing.difference_update(dates_observed_dt)
    
    if len(dates_missing) >= 1:
        
        first_missing_date = min(dates_missing)
    
        return(first_missing_date)
    else:
        
        return(None)

Checks for existing data in results folder for given AQY. If none exist, returns deployment date as first date of data needed. This is the parent function to _get deployment date, get_existing_dates_in_data, find_first_gap_in_data, and find_start_date_from_existing_.

In [18]:
def check_existing_get_start_date(aqy_id):

    existing_min_data_aqy = os.path.join('raw_minute_' + aqy_id + '.csv')
    existing_files_all_aqys = os.listdir('results_downloader_data/one_minute_data')
    
    deployment_date = get_deployment_date(aqy_id)
    
    ### USE BELOW TO FIGURE OUT SOME KIND OF CONDITIONAL EXECUTION FOR SAC DATA VS. DEPLOYMENT DATA
    return(deployment_date - datetime.timedelta(days = 180))
    
    if existing_min_data_aqy in existing_files_all_aqys:  
        
        existing_dates = get_existing_dates_in_data(os.path.join('results_downloader_data/one_minute_data', existing_min_data_aqy))
        first_data_gap = find_first_data_gap(existing_dates)                                         
        
        if first_data_gap is None:
            
            first_day_needed = 'None - up to date'
            
        else:
            
            first_day_needed = max(deployment_date, first_data_gap)
        
        return(first_day_needed)
         
    else:
        
        return(deployment_date)

Gets all necessary query dates (start of week and end of week) based on start date returned in above functions. Formats in all needed formats.

In [8]:
def get_all_query_dates(start_date):
    
    yesterday = datetime.date.today() - datetime.timedelta(days = 1)
    
    desired_dates = {'start_date_dt': start_date, 
                     'end_date_dt': None, 
                     'start_date_str': None,
                     'end_date_str': None
                    }
    
    desired_dates['end_date_dt'] = min(desired_dates['start_date_dt'] + datetime.timedelta(days = 6), yesterday)
    
    desired_dates['start_date_str'] = desired_dates['start_date_dt'].strftime('%Y-%m-%d')
    desired_dates['end_date_str'] = desired_dates['end_date_dt'].strftime('%Y-%m-%d')
    
    return(desired_dates)

Requests AQY data between the start and end date given for the requested averaging time.

In [9]:
def create_request_url(aqy_id, start_date, end_date):
    
    request_url_base = 'https://api.cloud.aeroqual.com/V2/'
    request_url_project = 'organisations/PSE%20Healthy%20Energy/projects/richmond_air/data?'
    request_url_id = 'instruments/{}/data?'.format(str(aqy_id.replace(' ', '%20')))
    request_url_dates = 'averagingPeriod=1&from={}%2000%3A00%3A00&to={}%2023%3A59%3A59&includeDiagnostics=true&rawValues=true&utc=false&includeSensorsWithNoData=true'.format(start_date, end_date)
    
    if aqy_id == 'Project':
        request_url = request_url_base + request_url_project + request_url_dates
    else:
        request_url = request_url_base + request_url_id + request_url_dates
    
    return(request_url)

In [10]:
def get_aqy_data_for_week(aqc_login, request_url): 
    
    aqy_data = requests.get(request_url, cookies = aqc_login.cookies)
    request_status = str(aqy_data)[-5:-2]
    
    if request_status != '200':
        print('Request status:', aqy_data)
        return(aqy_data.content)
    
    aqy_data_json = aqy_data.json()
    
    if 'Data' in aqy_data_json['Instruments'][0].keys():
        aqy_data_df = pd.json_normalize(aqy_data_json['Instruments'][0]['Data'])
    
        return(aqy_data_df)
    
    else:
        return(pd.DataFrame())

In [11]:
def initialize_results(aqy_id):
    
    existing_results_aqy = 'raw_minute_' + aqy_id + '.csv'
    existing_results_folder = 'results_downloader_data/one_minute_data'
    
    if existing_results_aqy in os.listdir(existing_results_folder):
        existing_data_df = pd.read_csv(os.path.join(existing_results_folder, existing_results_aqy))
        
        return(existing_data_df)
    
    else:
        blank_df = pd.DataFrame()
        
        return(blank_df)

Starts querying the API on the start date generated through the first steps. Subsequently adds seven days onto query start date and end date until reaching yesterday. Appends requested data to existing data.

In [19]:
def get_aqy_data_for_full_deployment(aqc_login, aqy_id):
    
    date_to_start = check_existing_get_start_date(aqy_id)
    
    if date_to_start == 'None - up to date': 
        return(print('Moving on to next AQY - data already up to date'))
    else:
        query_dates = get_all_query_dates(date_to_start)
    
    aqy_data_full = initialize_results(aqy_id)
    
    ### USE THE BELOW TO CRAFT CONDITIONAL EXECUTION FOR SAC DOWNLOAD VS. DEPLOYMENT DOWNLOAD
    while query_dates['start_date_dt'] < get_deployment_date(aqy_id):
    #while query_dates['start_date_dt'] < datetime.date.today() - datetime.timedelta(days = 1):
        print(query_dates['start_date_dt'], query_dates['end_date_dt'])
        
        request_url = create_request_url(aqy_id, query_dates['start_date_str'], query_dates['end_date_str'])
        aqy_data = get_aqy_data_for_week(aqcloud_login, request_url)
    
        query_dates = get_all_query_dates(query_dates['start_date_dt'] + datetime.timedelta(days = 7))
        
        if aqy_data.shape[1] > 1:
            aqy_data_full = pd.concat([aqy_data_full, aqy_data])
    
    aqy_data_full['ID'] = aqy_id
    aqy_data_full.drop_duplicates(inplace = True)
    
    print('Data downloaded from cloud and appended to existing file')
    
    ### GET RID OF _SAC
    path_to_new_data = 'results_downloader_data/one_minute_data/sac_raw_minute_{}.csv'.format(aqy_id)
    aqy_data_full.to_csv(path_to_new_data, index = False)
    
    return(aqy_data_full)

### Put it Together

In [20]:
aqcloud_login = create_aqc_login('lhill@psehealthyenergy.org', 'R1chmond@ir')

#aqys_available_list = get_instruments_available(aqcloud_login)
#aqys_available_list.remove('AQY BB-896')
#aqys_available_list.remove('AQY BB-803')

aqys_available_list = ['AQY BB-633', 'AQY BB-642']

for aqy in aqys_available_list:
    print('REQUESTING DATA FOR', aqy)
    get_aqy_data_for_full_deployment(aqcloud_login, aqy)

Login status: <Response [200]>
REQUESTING DATA FOR AQY BB-633
2019-12-06 2019-12-12
2019-12-13 2019-12-19
2019-12-20 2019-12-26
2019-12-27 2020-01-02
2020-01-03 2020-01-09
2020-01-10 2020-01-16
2020-01-17 2020-01-23
2020-01-24 2020-01-30
2020-01-31 2020-02-06
2020-02-07 2020-02-13
2020-02-14 2020-02-20
2020-02-21 2020-02-27
2020-02-28 2020-03-05
2020-03-06 2020-03-12
2020-03-13 2020-03-19
2020-03-20 2020-03-26
2020-03-27 2020-04-02
2020-04-03 2020-04-09
2020-04-10 2020-04-16
2020-04-17 2020-04-23
2020-04-24 2020-04-30
2020-05-01 2020-05-07
2020-05-08 2020-05-14
2020-05-15 2020-05-21
2020-05-22 2020-05-28
2020-05-29 2020-06-04
Data downloaded from cloud and appended to existing file
REQUESTING DATA FOR AQY BB-642
2019-12-06 2019-12-12
2019-12-13 2019-12-19
2019-12-20 2019-12-26
2019-12-27 2020-01-02
2020-01-03 2020-01-09
2020-01-10 2020-01-16
2020-01-17 2020-01-23
2020-01-24 2020-01-30
2020-01-31 2020-02-06
2020-02-07 2020-02-13
2020-02-14 2020-02-20
2020-02-21 2020-02-27
2020-02-28 202

### Script to collect Sacramento Co-location Data

In [15]:
def get_sac_data():
    
    aqcloud_login = create_aqc_login('lhill@psehealthyenergy.org', 'R1chmond@ir')

    aqys_available_list = get_instruments_available(aqcloud_login)
    aqys_available_list.remove('AQY BB-896')

    aqys_available_df = pd.DataFrame({'ID': aqys_available_list})
    aqys_available_df['start_date'] = aqys_available_df['ID'].apply(check_existing_get_start_date)
    first_date_needed = min(set(aqys_available_df['start_date']))
    query_dates = get_all_query_dates(first_date_needed)

    aqy_data_full = initialize_results('Project')

    while query_dates['start_date_dt'] < datetime.date.today() - datetime.timedelta(days = 366):
        print(query_dates['start_date_dt'], query_dates['end_date_dt'])

        request_url = create_request_url('Project', query_dates['start_date_str'], query_dates['end_date_str'])
        aqy_data = get_aqy_data_for_week(aqcloud_login, request_url)

        query_dates = get_all_query_dates(query_dates['start_date_dt'] + datetime.timedelta(days = 7))

        if aqy_data.shape[1] > 1:
            aqy_data_full = pd.concat([aqy_data_full, aqy_data])

In [16]:
def get_colocation_minute_data():
    aqcloud_login = create_aqc_login('lhill@psehealthyenergy.org', 'R1chmond@ir')

    #aqys_available_list = get_instruments_available(aqcloud_login)
    #aqys_available_list.remove('AQY BB-896')
    
    aqys_available_lst = ['AQY BB-633', 'AQY BB-642']
    
    for aqy in aqys_available_list:

        colocation_start = datetime.date(2019, 7, 15)
        query_dates = get_all_query_dates(colocation_start)
        colocation_end = get_deployment_date(aqy)

        aqy_data_full = pd.DataFrame()

        while query_dates['start_date_dt'] < colocation_end:

            aqy_data = get_aqy_data_for_week(aqcloud_login, aqy, query_dates['start_date_str'], query_dates['end_date_str'])

            print('Requested data for', aqy, 'from', query_dates['start_date_dt'], 'to', query_dates['end_date_dt'])

            query_dates = get_all_query_dates(query_dates['start_date_dt'] + datetime.timedelta(days = 7))

            if aqy_data.shape[1] > 1:
                aqy_data_full = pd.concat([aqy_data_full, aqy_data])

        aqy_data_full['ID'] = aqy

        aqy_data_full.to_csv(os.path.join('results', 'sac_colocation', aqy + '.csv'), index = False)

In [17]:
get_colocation_minute_data()

Login status: <Response [200]>


TypeError: get_aqy_data_for_week() takes 2 positional arguments but 4 were given

#Finds the date to start querying the data based on whether there are missing days throughout the data and whether the most recent timestamp is yesterday. Only called within _check_existing_get_start_date_ if there are existing data. Otherwise, deployment date is used.

def find_start_date_from_existing(existing_file_path):
    
    one_day_difference = datetime.timedelta(days = 1)
    
    existing_dates = get_existing_dates_in_data(existing_file_path)  
    
    days_after_gaps = existing_dates[existing_dates['time_dif'] != one_day_difference]
    number_of_gaps = days_after_gaps.shape[0]
    day_after_last_timestamp = existing_dates['date_observed'][-1:] + one_day_difference
    
    yesterday = datetime.date.today() - one_day_difference
    
    if number_of_gaps == 0:
        
        if day_after_last_timestamp == yesterday:
        
            print('Go analyze data! This AQY is already up-to-date')

            return('Up-to-date')
    
        else:         
            print('No gaps in data, but not quite up to date. Requesting  data starting at:', str(day_after_last_timestamp))
        
            return(day_after_last_timestamp)
        
    else:
        
        missing_day = find_first_gap_in_data(days_after_gaps, existing_dates)
        print('Data missing starting at ' + str(missing_day) + ': requesting from this date forward')
            
        return(missing_day)

#Creates a list of unique dates in the data. Identifies days that are >1 day after the preceeding day and labels as a gap. Feeds into _find_date_to_start_ if gaps are identified.

def look_for_first_gap_in_existing(days_after_gaps, existing_dates):
    
    day_after_first_gap = days_after_gaps['date_observed'].iloc[0]
            
    all_data_before_first_gap = existing_dates[existing_dates['date_observed'] < day_after_first_gap]
            
    day_before_first_gap = all_data_before_first_gap['date_observed'].iloc[-1]
        
    missing_day = day_before_first_gap + one_day_difference
    
    return(missing_day)

def get_sensors_available(aqc_login, aqy_id):
    
    aqy_request_url = 'https://cloud.aeroqual.com/api/instrument/{}'.format(aqy_id)
    
    available_aqy_data = requests.get(aqy_request_url, cookies = aqc_login.cookies)
    
    if str(available_aqy_data)[-5:-2] != '200':
        print('Request for', aqy_id, 'status:', available_aqy_data)
        
        return(available_aqy_data)
    else:
        
        aqy_metadata = available_aqy_data.json()
        
        aqy_sensors = [sensor['name'] for sensor in aqy_metadata['sensors']]
        
        return(aqy_sensors)

def get_data_by_date_network(aqc_login, list_of_aqys, start_date, end_date, avg_time):
    
    print('Requesting network data for range:', start_date, end_date)
    
    network_data_df = pd.DataFrame()
    
    for aqy in list_of_aqys:
        
        aqy_data_json = get_data_by_aqy(aqc_login, aqy, start_date, end_date, avg_time)

        aqy_data_df = pd.json_normalize(aqy_data_json['data'])
        aqy_data_df['ID'] = aqy
        
        if aqy_data_df.shape[1] > 1:
            network_data_df = network_data_df.append(aqy_data_df)
    
    return(network_data_df)